In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Add
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from keras.layers.merge import concatenate, add

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
dataset = pd.read_csv('/content/drive/My Drive/dataset/bbc-text.csv')

In [6]:
dataset.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [0]:
X = dataset.iloc[:,-1]

In [8]:
X.head()

0    tv future in the hands of viewers with home th...
1    worldcom boss  left books alone  former worldc...
2    tigers wary of farrell  gamble  leicester say ...
3    yeading face newcastle in fa cup premiership s...
4    ocean s twelve raids box office ocean s twelve...
Name: text, dtype: object

In [0]:
Y = dataset.iloc[:,0]

In [10]:
dataset['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [0]:
lb = LabelEncoder()
temp = lb.fit_transform(Y)
y = to_categorical(temp)

In [12]:
y[0]

array([0., 0., 0., 0., 1.], dtype=float32)

In [13]:
text = dataset['text']
tokenizer = Tokenizer(
    num_words = None,
    char_level=False,
    lower=True,
    split = ' ',
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
print(sequences[0])
max_len = 0
for i in sequences:
  l = len(i)
  if max_len < l:
    max_len = l
    
print(max_len)

padded_sequences = pad_sequences(sequences, maxlen=max_len)

[176, 264, 6, 1, 1252, 3, 1315, 17, 126, 1216, 838, 5277, 162, 1168, 4339, 4, 235, 272, 4201, 1440, 76, 1, 1407, 1685, 1, 117, 47, 1044, 176, 23, 16, 6572, 422, 6, 172, 79, 67, 10, 9, 228, 2, 34, 3207, 1426, 35, 2599, 21, 1, 676, 516, 1373, 149, 6, 3042, 3125, 2, 1843, 130, 182, 48, 991, 23, 852, 50, 3, 125, 1049, 19534, 17, 1, 49, 752, 1, 1668, 1147, 4, 71, 511, 23, 16, 2085, 2, 1315, 875, 126, 601, 179, 1589, 2198, 1742, 223, 4, 427, 209, 2824, 2, 914, 5278, 4, 1182, 707, 50, 3, 1, 116, 4483, 52, 991, 3, 2660, 19, 38, 235, 4, 447, 272, 4201, 8861, 4, 5858, 182, 112, 150, 3296, 92, 1, 49, 7, 5538, 4, 1, 77, 7, 1492, 233, 553, 47, 2, 226, 1493, 159, 4665, 4, 587, 5279, 176, 1147, 66, 29, 164, 6573, 1, 160, 2199, 8, 140, 40, 8862, 176, 29, 25, 45, 88, 1217, 6, 2, 162, 1168, 176, 2056, 35, 25, 236, 247, 6, 517, 4, 1, 49, 24, 2958, 2, 114, 128, 6, 237, 102, 3, 1, 1316, 3, 162, 1168, 4484, 28, 85, 55, 47, 587, 5279, 179, 3208, 29, 55, 45, 2773, 52, 7022, 22, 509, 4, 1191, 8863, 1471, 713, 

In [0]:
num_words = vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
max_sequence_length = max_len
extra_conv = True
labels_index = 5

In [15]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            input_length=max_sequence_length)

sequence_input = Input(shape=(max_sequence_length,))

embedded_sequences = embedding_layer(sequence_input)
convs = []
filter_sizes = [4,5,6] # in the loop, first apply 3 as size, then 4 then 5

for filter_size in filter_sizes:
    l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
    #kernel is the filter
    l_pool = MaxPooling1D(pool_size=3)(l_conv)
    convs.append(l_pool)

l_merge = concatenate(convs, axis=1)

    
conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
pool = MaxPooling1D(pool_size=3)(conv)

if extra_conv==True:
    x = Dropout(0.5)(l_merge)  
else:
        # Original Yoon Kim model
    x = Dropout(0.5)(pool)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
    
preds = Dense(labels_index, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
x_complete = pd.DataFrame(padded_sequences)
x_complete.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4451,4452,4453,4454,4455,4456,4457,4458,4459,4460,4461,4462,4463,4464,4465,4466,4467,4468,4469,4470,4471,4472,4473,4474,4475,4476,4477,4478,4479,4480,4481,4482,4483,4484,4485,4486,4487,4488,4489,4490
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5,3814,17,5538,251,19544,35,507,47,55,159,1865,1147,12,921,971,4,168,707,58,182,3209,1,1473,1668,3,9797,42,1758,73,10,47,55,1044,74,29,164,66,29,164
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1710,4342,1236,330,52,8152,4,367,104,861,330,39,501,32,1743,658,9,186,2,63,68,201,4,60,294,1272,1,214,7028,1253,5,3044,2057,2200,15,19,4863,2477,30,7029
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,541,1,5284,1,3128,9,53,12,349,15,55,982,10,3376,21,33,267,1,2432,23,20,2,16,2602,165,1,346,3,10,6576,4,1,1966,3,1712,6,5,1192,95,1967
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,12641,4486,903,703,6203,903,7032,12642,903,19550,12637,903,4072,10967,903,3210,3595,903,15009,4865,903,2434,10971,903,15010,4206,8153,903,10965,5285,1360,2,16,379,12,746,3,461,512,325
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2244,21,3480,6578,183,49,1008,44,363,15019,52,1,12647,12648,3130,650,17,1,1568,1744,331,8160,11,5,15020,7035,650,5,9803,1376,6,1,48,651,331,2959,1,2302,15021,12649


In [17]:
y_complete = pd.DataFrame(y)
y_complete.head()

,0,1,2,3,4
0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0


In [18]:
x_complete.shape

(2225, 4491)

In [0]:
x_train = x_complete.iloc[:1800,]
y_train = y_complete.iloc[:1800,]
x_test = x_complete.iloc[1800:,]
y_test = y_complete.iloc[1800:,]

In [20]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]
hist = model.fit(x_train, y_train, epochs=10, callbacks=callbacks_list, validation_split=0.1, shuffle=True,batch_size = 128)




Train on 1620 samples, validate on 180 samples
Epoch 1/10





1620/1620 [==============================] - 301s 186ms/step - loss: 0.8793 - acc: 0.7514 - val_loss: 0.4912 - val_acc: 0.8000
Epoch 2/10
1620/1620 [==============================] - 294s 181ms/step - loss: 0.4952 - acc: 0.8002 - val_loss: 0.4944 - val_acc: 0.8000
Epoch 3/10
1620/1620 [==============================] - 300s 185ms/step - loss: 0.4873 - acc: 0.8005 - val_loss: 0.4814 - val_acc: 0.8000
Epoch 4/10
1620/1620 [==============================] - 301s 186ms/step - loss: 0.4489 - acc: 0.8189 - val_loss: 0.4169 - val_acc: 0.8422
Epoch 5/10
1620/1620 [==============================] - 297s 184ms/step - loss: 0.3625 - acc: 0.8470 - val_loss: 0.3231 - val_acc: 0.8589
Epoch 6/10
1620/1620 [==============================] - 296s 183ms/step - loss: 0.2770 - acc: 0.8789 - val_loss: 0.2482 - val_acc: 0.8978
Epoch 7/10
1620/1620 [==============================] - 292s 180ms/step - loss: 0.1676 - acc: 0.9346 - val_loss: 0.13

In [0]:
y_pred = model.predict(x_test)

In [41]:
result = model.evaluate(x_test,y_test)
result

425/425 [==============================] - 18s 43ms/step


[0.07815255165100098, 0.9755294213575475]

In [49]:
y_test.shape

(425, 5)

In [0]:
rounded_labels=np.argmax(np.array(y_test), axis=1)
#rounded_pred = np.argmax(y_pred,axis = 1)

In [51]:
rounded_labels.shape

(425,)

In [0]:
from sklearn.metrics import classification_report
report = classification_report(rounded_labels,rounded_pred)

In [55]:
print(report)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       103
           1       0.91      0.85      0.88        60
           2       0.93      0.93      0.93        82
           3       0.98      0.93      0.95        97
           4       0.96      0.99      0.98        83

    accuracy                           0.94       425
   macro avg       0.94      0.93      0.93       425
weighted avg       0.94      0.94      0.94       425

